# HOW TO USE Google Earth Engine (EE) and GEEMAP

* reference: https://github.com/gee-community/geemap/tree/master/examples 

## Getting Started


In [2]:
import geemap

You need to authenticate Earth Engine before using it. The package for the Earth Engine Python API is called earthengine-api, which should have been automatically installed by the geemap package. 

Your credential is stored in `/Users/USERNAME/.config/earthengine/credentials`

In [3]:
import ee
ee.Authenticate()

True

In [4]:
ee.Initialize(project='satellite-example')
# ee.Initialize()
#ee.Initialize(project='<PROJECT-ID>')

In [15]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [17]:
# Seoul 37.5652° N, 126.9774° E
Map = geemap.Map(center=[38, 126], zoom=4)
Map

Map(center=[38, 126], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tra…

In [18]:
Map.add_basemap("HYBRID")

In [20]:
# US coordinate 38.7946° N, 106.5348° W
Map = geemap.Map(center=[40, -100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [21]:
# https://www.usgs.gov/data/national-land-cover-database-nlcd-2019-products-ver-30-february-2024 
# https://www.mrlc.gov

Map = geemap.Map(center=[40, -100], zoom=4)
url = 'https://www.mrlc.gov/geoserver/mrlc_display/NLCD_2019_Land_Cover_L48/wms?'
Map.add_wms_layer(
    url=url,
    layers='NLCD_2019_Land_Cover_L48',
    name='NLCD 2019',
    format='image/png',
    attribution='MRLC',
    transparent=True,
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

## Earth DataType


In [25]:
# Load Earth Engine Image
# https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003
# The Shuttle Radar Topography Mission (SRTM, see Farr et al. 2007) digital elevation data

image = ee.Image('USGS/SRTMGL1_003')
image
image.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -32768,
    'max': 32767},
   'dimensions': [1296001, 417601],
   'crs': 'EPSG:4326',
   'crs_transform': [0.0002777777777777778,
    0,
    -180.0001388888889,
    0,
    -0.0002777777777777778,
    60.00013888888889]}],
 'version': 1641990767055141,
 'id': 'USGS/SRTMGL1_003',
 'properties': {'system:visualization_0_min': '0.0',
  'type_name': 'Image',
  'keywords': ['dem',
   'elevation',
   'geophysical',
   'nasa',
   'srtm',
   'topography',
   'usgs'],
  'thumb': 'https://mw1.google.com/ges/dd/images/SRTM90_V4_thumb.png',
  'description': '<p>The Shuttle Radar Topography Mission (SRTM, see <a href="https://onlinelibrary.wiley.com/doi/10.1029/2005RG000183/full">Farr\net al. 2007</a>)\ndigital elevation data is an international research effort that\nobtained digital elevation models on a near-global scale. This\nSRTM V3 product (SRTM Plus) is provided by NASA JP

In [27]:
# Map = geemap.Map(center=[21.79, 70.87], zoom=3)
Map = geemap.Map(center=[40, -100], zoom=3)
image = ee.Image('USGS/SRTMGL1_003')
vis_params = {
    'min': 0,
    'max': 6000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(image, vis_params, 'SRTM')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [33]:
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED

Map = geemap.Map()
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(83.277, 17.7009, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…

In [34]:
# filtering image collections

Map = geemap.Map()
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2022-01-01')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)
image = collection.median()

vis = {
    'min': 0.0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}

Map.setCenter(83.277, 17.7009, 12)
Map.addLayer(image, vis, 'Sentinel-2')
Map

Map(center=[17.7009, 83.277], controls=(WidgetControl(options=['position', 'transparent_bg'], position='toprig…

In [22]:
# Geometry

# Seoul 37.5652° N, 126.9774° E

# Define a Point object.
point = ee.Geometry.Point(37, 126)

# Define other inputs.
input_geom = ee.Geometry.BBox(125, 36, 127, 38)

# Apply the union method to the Point object.
point_union = point.union(right=input_geom, maxError=1)

# Print the result.
display('point.union(...) =', point_union)

# Display relevant geometries on the map.
m = geemap.Map(zoom=7)
m.set_center(126, 37)
m.add_layer(point, {'color': 'black'}, 'Geometry [black]: point')
m.add_layer(input_geom, {'color': 'blue'}, 'Parameter [blue]: input_geom')
m.add_layer(point_union, {'color': 'red'}, 'Result [red]: point.union')
m


'point.union(...) ='

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.union",
    "arguments": {
      "left": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Point",
          "arguments": {
            "coordinates": {
              "constantValue": [
                37,
                126
              ]
            }
          }
        }
      },
      "maxError": {
        "functionInvocationValue": {
          "functionName": "ErrorMargin",
          "arguments": {
            "value": {
              "constantValue": 1
            }
          }
        }
      },
      "right": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    125,
                    38
                  ],
                  [
                    125,
                    36
                  ],
                  [
                    127,
                    36
                  ],
                  [
                    127,
                    38
                  ]
                ]
              ]
            },
            "geodesic": {
              "constantValue": false
            }
          }
        }
      }
    }
  }
})

Map(center=[37, 126], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tra…

In [23]:
rectangle = ee.Geometry.Rectangle([120, 30, 125, 35])
m.addLayer(rectangle, {}, 'Rectangle')
m


Map(bottom=13143.0, center=[36.2176871222506, 126.29882812500001], controls=(WidgetControl(options=['position'…

In [29]:
# https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
feat = states.filter(ee.Filter.eq('NAME', 'Texas'))
Map.addLayer(feat, {}, 'Texas')
Map.centerObject(feat)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [30]:
texas = feat.first()
texas.toDictionary()

In [31]:
Map = geemap.Map()
states = ee.FeatureCollection('TIGER/2018/States')
fc = states.filter(ee.Filter.inList('NAME', ['California', 'Oregon', 'Washington']))
Map.addLayer(fc, {}, 'West Coast')
Map.centerObject(fc)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [32]:
region = Map.user_roi
if region is None:
    region = ee.Geometry.BBox(-88.40, 29.88, -77.90, 35.39)

fc = ee.FeatureCollection('TIGER/2018/States').filterBounds(region)
Map.addLayer(fc, {}, 'Southeastern U.S.')
Map.centerObject(fc)

In [33]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [34]:

Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
image = ee.Image().paint(states, 0, 3)
Map.addLayer(image, {'palette': 'red'}, "US States")
Map
     

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [35]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000080'}
Map.addLayer(states.style(**style), {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [36]:
Map = geemap.Map(center=[40, -100], zoom=4)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    states, column="NAME", palette=palette, layer_name="Styled vector", **vis_params
)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [41]:
from geemap.datasets import DATA

Map = geemap.Map(center=[40, -100], zoom=4)
dataset = ee.Image(DATA.USGS_GAP_CONUS_2011)
Map.addLayer(dataset, {}, 'GAP CONUS')
Map
     

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [42]:

from geemap.datasets import get_metadata

get_metadata(DATA.USGS_GAP_CONUS_2011)

HTML(value='\n            <html>\n            <body>\n                <h3>USGS GAP CONUS 2011</h3>\n          …

In [45]:
# Getting MetaData Image
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')

In [46]:
image.bandNames()

In [48]:
image.select('SR_B1').projection()
# EPSG:32610 >> coordinate

In [50]:
image.toDictionary()

In [51]:
props = geemap.image_props(image)
props

In [52]:
# Find the min, max for image
image = ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_044034_20220503')     

In [53]:
geemap.image_min_value(image)
geemap.image_max_value(image)

In [54]:
geemap.image_stats(image)

In [55]:
Map = geemap.Map(center=(40, -100), zoom=4)
dem = ee.Image('USGS/SRTMGL1_003')
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').select(
    ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
)
states = ee.FeatureCollection("TIGER/2018/States")
vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}
Map.addLayer(dem, vis_params, 'SRTM DEM')
Map.addLayer(
    landsat7,
    {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200, 'gamma': 2.0},
    'Landsat 7',
)
Map.addLayer(states, {}, "US States")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…